In [1]:
import os
import requests
import concurrent.futures
import cv2
import numpy as np
from PIL import Image, UnidentifiedImageError
import csv
from nudenet import NudeClassifier
# REFER IF NUDENET STOPS WORKING - https://stackoverflow.com/questions/70696761/python-nsfw-detection-module-nudenet-not-longer-working
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# STEP 1 : STORING RAW COLLECTION OF IMAGES IN DIRECTORY RAW_DATA & INITIATE EMPTY LIST
directory = 'raw_data'
file_contents = []

'''

STOPPED DOWNLOADING AT 
http://i.imgur.com/e3P6L.jpg

'''

# STEP 2 : COMPARE IMGUR IMAGE WITH TEMPLATE
img1 = cv2.imread('image_not_available.jpg')

# STEP 2 : CLEANING RAW DATA AND REMOVING NOISE & GARBAGE INPUTS
dir_path = 'data'

# STEP 3 : PROVIDE ML MODEL WITH DATA & INITIATE NUDECLASSIFIER
input_path = r"C:\Users\Aditya\Documents\GitHub\obscene_img_ml\data"
classifier = NudeClassifier()

In [ ]:
# FUNCTION DEFINITION

# FUNCTION 1.1 - DOWNLOAD_IMAGE - URL
def download_image(url):
    try:
        response = requests.get(url, timeout=5)
        print(url)
        url_list = url.split("/")
        url_last = url_list[-1]
        url_last = url_last.replace(":","_")
        url_last = url_last.replace("-","_")
        url_last = url_last.replace(".","_")
        url_last = url_last.replace(";","_")
        url_last = url_last.replace("?","_")
        url_last = url_last.replace("=","_")
        url_last = url_last.replace("!","_")
        with open("data/"+str(url_last)+".jpg", 'wb') as f:
            f.write(response.content)
    except:
        print("error")
        pass

# FUNCTION 2.1 - REMOVE EMPTY IMGUR TEMPLATE IMAGES
def compare_images(img2, loc):
    try:
        mse = np.mean((img1 - img2) ** 2)
        if mse < 10:
            os.remove(loc)
            print("deleted, score: ", str(mse))
        else:
            print("The images are different")
    except ValueError:
        pass

# FUNCTION 2.2 - REMOVE MP4 AND GIF FILES
def remove_mp4(img2):
    imgval = img2[:-4]
    img2_name = imgval.split("_")
    if "mp4" == img2_name[-1] or "gif" == img2_name[-1]:
        os.remove(img2)
        print("deleted mp4")

# FUNCTION 2.3 - REMOVE CORRUPTED IMAGES
def corrupted_img(img2):
    try:
        with Image.open(img2) as img:
            img.verify()
    except (IOError, SyntaxError) as e:
        os.remove(img2)
        print("deleted corrupted image")

# FUNCTION 3.1 - BEGIN ML MODEL CLASSIFICATION
def checkval(classifier, root, file_name):
    value = os.path.join(root, file_name)
    try:
        dict_val = classifier.classify(value)
        if dict_val[value]["unsafe"] > dict_val[value]["safe"]:
            classification = "Obscene"
        else:
            classification = "SFW"

        with open("output.csv", mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([value, classification])
    except Exception as e:
        classification = "ERROR"
        with open("output.csv", mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([value, classification])

In [ ]:
# STEP 1.1 - DOWNLOAD IMAGES FROM URLS & RENAME IMAGES
# CALL FUNCTION 1.1

for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            contents = f.read()
            data = contents.split('\n')
            file_contents.extend(data)

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(download_image, url) for url in file_contents]
    concurrent.futures.wait(futures)

In [ ]:
# STEP 2.1 - REMOVE EMPTY IMGUR TEMPLATE IMAGES
# CALL FUNCTION 2.1

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            img2 = cv2.imread(os.path.join(root, file))
            loc = os.path.join(root, file)
            executor.submit(compare_images, img2, loc)


In [ ]:
# STEP 2.2 - REMOVE MP4 AND GIF FILES
# CALL FUNCTION 2.2

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            img2 = os.path.join(root, file)
            executor.submit(remove_mp4, img2)

In [ ]:
# STEP 2.3 - REMOVE CORRUPTED IMAGES
# CALL FUNCTION 2.3

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            img2 = os.path.join(root, file)
            executor.submit(corrupted_img, img2)

In [ ]:
# STEP 3.1 - BEGIN ML MODEL CLASSIFICATION
# CALL FUNCTION 3.1

with open("output.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Path", "Classification"])

for root, dirs, files in os.walk(input_path):
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        futures = [executor.submit(checkval, classifier, root, file_name) for file_name in files]
        concurrent.futures.wait(futures)

In [ ]:
# STEP 4.1 - EXTRACT ANALYSIS FROM CLASSIFICATION

%matplotlib inline
df = pd.read_csv('output.csv')
counts = df['Classification'].value_counts()
print(counts)
counts.plot(kind='bar')
plt.show()

In [ ]:
# STEP 5.1 - ERROR HANDLING
 
for index, row in df.iterrows():
    if row['Classification'] == 'ERROR':
        print(row['File Path'])